In [1]:
%load_ext autoreload
%autoreload 2
from trainer import LitModel
import torch 
from shared_modules.data_module_all import DataModule
from shared_modules.utils import load_config
from shared_modules.plotting import plot_metrics, plot_confusion, plot_difference
from shared_modules.torch_metrics import PicaiMetric
from shared_modules.post_transforms import get_post_transforms
from monai.transforms import AsDiscrete
from tqdm import tqdm
from monai.metrics import DiceMetric
from monai.data import decollate_batch

<frozen importlib._bootstrap_external>:1328: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


If you have questions or suggestions, feel free to open an issue at https://github.com/DIAGNijmegen/picai_eval



Please cite the following paper when using Report Guided Annotations:

Bosma, J.S., et al. "Semi-supervised learning with report-guided lesion annotation for deep learning-based prostate cancer detection in bpMRI" to be submitted


If you have questions or suggestions, feel free to open an issue at https://github.com/DIAGNijmegen/Report-Guided-Annotation



In [2]:
SAVE_PROB_MAPS = False
SAVE_PREDS = False

config = load_config("config.yaml")
config.data.data_dir = "../../../data/"
config.data.json_list = "../../../json_datalists/picai/all_samples.json"
gpu = 0
config.gpus = [gpu]
config.cache_rate = 1.0
config.transforms.label_keys=["pca", "prostate"]
config.transforms.crop_key = "prostate"
# config.transforms.image_keys = ["t2w", "adc", "hbv"]
config.transforms.image_keys = ["image"]
label_key = config.transforms.label_keys[0]

In [3]:
weights_folder = "../../../gc_algorithms/base_container/models/umamba_mtl/weights/"
models = []

for i in range(5):
    models.append(LitModel.load_from_checkpoint(f"{weights_folder}f{i}.ckpt", config=config, map_location=f"cuda:{gpu}"))
    # disable randomness, dropout, etc...
    models[-1].eval()
    models[-1].to(gpu)

In [4]:
dm = DataModule(
    config=config,
)

dm.setup("test")
dl = [next(iter(dm.test_dataloader()))]

monai.transforms.spatial.dictionary Orientationd.__init__:labels: Current default value of argument `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` was changed in version None from `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` to `labels=None`. Default value changed to None meaning that the transform now uses the 'space' of a meta-tensor, if applicable, to determine appropriate axis labels.
Loading dataset: 100%|██████████| 1499/1499 [03:01<00:00,  8.27it/s]


In [5]:
prob_map_post_transforms = get_post_transforms(key="prob", 
                    orig_key=label_key,
                    orig_transforms=dm.transforms["test"],
                    out_dir=f"output/prob/",
                    keep_n_largest_components=0,
                    output_postfix="",
                    output_dtype="float32",
                    save_mask=SAVE_PROB_MAPS) # True

pca_post_transforms = get_post_transforms(key="pca", 
                    orig_key=label_key,
                    orig_transforms=dm.transforms["test"],
                    out_dir=f"output/pred/",
                    keep_n_largest_components=0,
                    output_postfix="",
                    output_dtype="float32",
                    save_mask=SAVE_PREDS) # True

In [7]:
from captum.attr import Occlusion, FeatureAblation, LayerGradCam
from captum.attr import visualization as viz

model = models[0]

for image in tqdm(dl):
    x = image["image"].to(0)
    logits = model(x)
    pred = AsDiscrete(argmax=True)(logits[0])[None].cpu()

def agg_segmentation_wrapper(image):
    pass




100%|██████████| 1/1 [00:10<00:00, 10.14s/it]


In [8]:
import torch
from concurrent.futures import ThreadPoolExecutor


# After loading models, distribute them across GPUs
gpus = [0, 1, 2, 3, 4]
dsc_fn = DiceMetric(include_background=False, reduction="mean")
picai_metric_fn = PicaiMetric()

all_probs = []
all_gts = []

# Use the first GPU for metric computation
metric_gpu = gpus[0]

for batch in tqdm(dl):
    with torch.no_grad():
        x = batch["image"]
        
        preds = []
        probs = []
        
        def run_model(fold, model, gpu, x):
            x_gpu = x.to(gpu)
            logits = model.inferer(x_gpu)
            pred = AsDiscrete(argmax=True)(logits[0])[None].cpu()  # Move back to CPU
            prob = torch.sigmoid(logits[0,1])[None][None].cpu()
            return pred, prob
        
        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = [
                executor.submit(run_model, fold, model, gpus[fold], x) 
                for fold, model in enumerate(models)
            ]
            for future in futures:
                pred, prob = future.result()
                preds.append(pred)
                probs.append(prob)
        
    batch["prob"] = torch.mean(torch.stack(probs), dim=0)
    
    # Reverts back to original size
    batch["prob"] = [prob_map_post_transforms(i)["prob"] for i in decollate_batch(batch)][0][None]
    batch["pca"] = [pca_post_transforms(i)["pca"] for i in decollate_batch(batch)][0][None]
    batch["pred"] = (batch["prob"] > 0.5).float()
     
    all_probs.append(batch["prob"][0,0,...].cpu().numpy())
    all_gts.append(batch["pca"][0,0,...].cpu().numpy())
    
    # Keep both on same device for metric computation
    dsc_fn(y_pred=batch["pred"].to(metric_gpu), y=batch["pca"].to(metric_gpu))
        
dsc_metrics = dsc_fn.aggregate("none")

  0%|          | 0/1 [00:09<?, ?it/s]


AcceleratorError: CUDA error: invalid device ordinal
GPU device may be out of range, do you have enough GPUs?
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [9]:
dsc_fn = DiceMetric(include_background=False, reduction="mean")
picai_metric_fn = PicaiMetric()

all_probs = []
all_gts = []


for batch in tqdm(dl):
    with torch.no_grad():
        x = batch["image"].to(gpu)

        preds = []
        probs = []
        
        for fold, model in enumerate(models):
            logits = model.inferer(x)
            preds.append(AsDiscrete(argmax=True)(logits[0])[None])
            probs.append(torch.sigmoid(logits[0,1])[None][None])
            
        
    batch["prob"] = torch.mean(torch.stack(probs), dim=0)
    
    # Reverts back to original size
    batch["prob"] = [prob_map_post_transforms(i)["prob"] for i in decollate_batch(batch)][0][None]
    batch["pca"] = [pca_post_transforms(i)["pca"] for i in decollate_batch(batch)][0][None]
    batch["pred"] = (batch["prob"] > 0.5).float()
     
    all_probs.append(batch["prob"][0,0,...].cpu().numpy())
    all_gts.append(batch["pca"][0,0,...].cpu().numpy())
    
    dsc_fn(y_pred=batch["pred"], y=batch["pca"].to(gpu))
        
dsc_metrics = dsc_fn.aggregate("none")

100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


In [10]:
import picai_eval
from report_guided_annotation import extract_lesion_candidates

metrics = picai_eval.evaluate(
            y_det=all_probs,
            y_true=all_gts,
            y_det_postprocess_func=lambda pred: extract_lesion_candidates(pred, threshold="dynamic")[0],
            y_true_postprocess_func=lambda y: y,
            num_parallel_calls=10
        )

metrics

Metrics(auroc=nan%, AP=-0.00%, 1 cases, 0 lesions)

In [ ]:
plot_metrics(metrics,56)

In [ ]:
plot_confusion(metrics, 56, threshold=0.5)